In [1]:
from src.model import get_dataset, Trainer, RNN, LSTM, CNN, CNN1d, BERTGRUSentiment
import torch.optim as optim
import torch.nn as nn
from transformers import BertTokenizer, BertModel

# RNN

In [2]:
INPUT_DIM = 1 # len(dataset["TEXT"].vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [3]:
trainer = Trainer(model, dump_path="/home/jupyter/wl_research/wl_research_challenge/dump_path")

In [4]:
trainer.compile(optimizer = "SGD", criterion = "BCEWithLogitsLoss", seed = 1234, batch_size = 64, max_vocab_size = 25000)

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000
Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2
The model has 2,592,105 trainable parameters



In [6]:
print(vars(trainer.dataset["train_data"].examples[0]))

{'text': ['How', 'can', 'a', 'movie', 'with', 'Amy', ',', 'Posey', 'and', 'Raechel', 'have', 'NOTHING', 'funny', 'in', 'it', '?', 'Believe', 'it', 'or', 'not', "'", 'House', 'Bunny', "'", 'did', 'this', 'better', 'and', 'funnier', '.', 'Hopefully', 'the', 'principals', 'had', 'a', 'good', 'holiday', 'and', 'got', 'some', 'money', '-', 'this', 'movie', 'is', 'an', 'embarrassment', 'to', 'all', 'of', 'them', '.', 'It', 'is', 'a', 'cliché', 'from', 'beginning', 'to', 'end', '.', 'Clichés', 'can', 'work', 'well', 'with', 'a', 'script', ',', 'or', 'at', 'least', 'an', 'idea', '.', 'This', 'movie', 'does', 'nothing', 'but', 'use', 'cliché', 'after', 'cliché', 'rather', 'than', 'ideas', 'or', 'script', '.', 'It', 'uses', 'the', 'preexisting', 'persona', "'s", 'of', 'the', 'actresses', 'rather', 'than', 'develop', 'characters', '.', 'Bad', ',', 'sad', ',', 'and', 'rubbish', '.', 'Now', 'I', 'apparently', 'have', 'to', 'have', 'ten', 'lines', 'of', 'text', 'for', 'a', 'comment', '.', 'Really', 

Why is the vocab size 25002 and not 25000? One of the addition tokens is the `<unk>` token and the other is a `<pad>` token.

When we feed sentences into our model, we feed a _batch_ of them at a time, i.e. more than one at a time, and all sentences in the batch need to be the same size. Thus, to ensure each sentence in the batch is the same size, any shorter than the longest within the batch are padded.

We can also view the most common words in the vocabulary and their frequencies.

In [7]:
print(trainer.dataset["TEXT"].vocab.freqs.most_common(20))

[('the', 202820), (',', 192918), ('.', 164754), ('and', 109435), ('a', 109381), ('of', 101102), ('to', 93825), ('is', 76552), ('in', 61403), ('I', 54228), ('it', 53519), ('that', 49356), ('"', 44224), ("'s", 43374), ('this', 42438), ('-', 37284), ('/><br', 35417), ('was', 34920), ('as', 30245), ('with', 29869)]


We can also see the vocabulary directly using either the `stoi` (string to int) or `itos` (int to string) method.

In [11]:
print(trainer.dataset["TEXT"].vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


We can also check the labels, ensuring 0 is for negative and 1 is for positive.

In [12]:
print(trainer.dataset["LABEL"].vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [5]:
trainer.train(max_epochs = 50, improving_limit = 2, dump_id = "")

Epoch: 01 | Epoch Time: 0m 35s
	Train Loss: 0.708 | Train Acc: 49.45%
	 Val. Loss: 0.700 |  Val. Acc: 50.36%
	New best validation score
Epoch: 02 | Epoch Time: 0m 35s
	Train Loss: 0.696 | Train Acc: 49.69%
	 Val. Loss: 0.696 |  Val. Acc: 50.60%
	New best validation score
Epoch: 03 | Epoch Time: 0m 36s
	Train Loss: 0.696 | Train Acc: 49.42%
	 Val. Loss: 0.697 |  Val. Acc: 50.44%
	Not a better validation score (0 / 2).
Epoch: 04 | Epoch Time: 0m 35s
	Train Loss: 0.696 | Train Acc: 49.90%
	 Val. Loss: 0.695 |  Val. Acc: 51.12%
	New best validation score
Epoch: 05 | Epoch Time: 0m 35s
	Train Loss: 0.696 | Train Acc: 50.05%
	 Val. Loss: 0.701 |  Val. Acc: 49.74%
	Not a better validation score (0 / 2).
Epoch: 06 | Epoch Time: 0m 36s
	Train Loss: 0.696 | Train Acc: 50.41%
	 Val. Loss: 0.697 |  Val. Acc: 50.79%
	Not a better validation score (1 / 2).
Epoch: 07 | Epoch Time: 0m 35s
	Train Loss: 0.696 | Train Acc: 49.78%
	 Val. Loss: 0.696 |  Val. Acc: 49.62%
	Not a better validation score (2 / 

In [7]:
trainer.test()

Test Loss: 0.689 | Test Acc: 54.04%


In [44]:
predict = trainer.get_predict_sentiment() #.predict_sentiment()

In [45]:
print(predict("What can you do with the fire ?"))

0.8343528509140015


In [46]:
print(predict("NLP is awesome !"))

0.5052047967910767


In [47]:
print(predict("Corona-virus has killed the whole world, the economy is suffering, empires have fallen, what a disaster !"))
print(predict("Gone has killed the whole world, the economy is suffering, empires have fallen, what a disaster !"))

0.8909756541252136
0.19438283145427704


# LSTM

In [13]:
INPUT_DIM = 25002 # will be update to len(dataset["TEXT"].vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = 1 # will be update to dataset["TEXT"].vocab.stoi[dataset["TEXT"].pad_token]

model = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

In [14]:
trainer = Trainer(model, dump_path="/home/jupyter/wl_research/wl_research_challenge/dump_path")

In [21]:
#trainer.compile(optimizer = "Adam", criterion = "BCEWithLogitsLoss", seed = 1234, batch_size = 64, max_vocab_size = 25000)
trainer.compile(optimizer = "Adam", criterion = "BCEWithLogitsLoss", seed = 1234, batch_size = 1, max_vocab_size = 25000)

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000
Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2
pretrained_embeddings.shape torch.Size([25002, 100])
self.model.embedding.weight.data tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.2295, -0.4421,  0.4331,  ..., -0.3084, -0.3175,  0.3562],
        [-0.0046, -0.4325, -0.3023,  ...,  0.3291, -0.1993, -0.2312],
        [-0.0212,  0.5229,  0.7983,  ..., -0.4114,  0.0744,  0.6785]],
       device='cuda:0')
The model has 4,810,857 trainable parameters



In [22]:
trainer.train(max_epochs = 50, improving_limit = 2, dump_id = "")

Epoch: 01 | Epoch Time: 29m 47s
	Train Loss: 0.692 | Train Acc: 51.58%
	 Val. Loss: 0.691 |  Val. Acc: 55.29%
	New best validation score


KeyboardInterrupt: 

In [ ]:
trainer.test()

In [24]:
predict = trainer.get_predict_sentiment() #.predict_sentiment()

In [25]:
print(predict(sentence = "What can you do with the fire ?"))

0.49243149161338806


# CNN

In [26]:
INPUT_DIM =   100 # will be update to len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = 1 # will be update to TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [27]:
trainer = Trainer(model, dump_path="/home/jupyter/wl_research/wl_research_challenge/dump_path")

In [28]:
trainer.compile(optimizer = "Adam", criterion = "BCEWithLogitsLoss", seed = 1234, batch_size = 64, max_vocab_size = 25000)

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000
Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2
The model has 2,620,801 trainable parameters



In [29]:
trainer.train(max_epochs = 50, improving_limit = 2, dump_id = "")

Epoch: 01 | Epoch Time: 0m 49s
	Train Loss: 0.729 | Train Acc: 49.66%
	 Val. Loss: 0.694 |  Val. Acc: 50.26%
	New best validation score
Epoch: 02 | Epoch Time: 0m 50s
	Train Loss: 0.722 | Train Acc: 50.40%
	 Val. Loss: 0.693 |  Val. Acc: 51.57%
	New best validation score
Epoch: 03 | Epoch Time: 0m 50s
	Train Loss: 0.719 | Train Acc: 50.33%
	 Val. Loss: 0.691 |  Val. Acc: 52.64%
	New best validation score
Epoch: 04 | Epoch Time: 0m 50s
	Train Loss: 0.715 | Train Acc: 50.72%
	 Val. Loss: 0.690 |  Val. Acc: 53.66%
	New best validation score
Epoch: 05 | Epoch Time: 0m 49s
	Train Loss: 0.711 | Train Acc: 50.44%
	 Val. Loss: 0.689 |  Val. Acc: 54.38%
	New best validation score
Epoch: 06 | Epoch Time: 0m 50s
	Train Loss: 0.704 | Train Acc: 51.82%
	 Val. Loss: 0.689 |  Val. Acc: 54.02%
	Not a better validation score (0 / 2).
Epoch: 07 | Epoch Time: 0m 50s
	Train Loss: 0.706 | Train Acc: 50.83%
	 Val. Loss: 0.688 |  Val. Acc: 55.23%
	New best validation score
Epoch: 08 | Epoch Time: 0m 50s
	Tra

In [21]:
trainer.test()

Test Loss: 0.748 | Test Acc: 49.99%


In [22]:
predict = trainer.get_predict_sentiment() #.predict_sentiment()

In [23]:
print(predict(sentence = "What can you do with the fire ?"))

0.304843932390213


## BERT

In [2]:
bert = BertModel.from_pretrained('bert-base-uncased')

In [3]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25
model = BERTGRUSentiment(bert, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

In [4]:
trainer = Trainer(model, dump_path="/home/jupyter/wl_research/wl_research_challenge/dump_path")

The model has 112,241,409 total parameters
The model has 2,759,169 trainable parameters ...
rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias



In [5]:
trainer.compile(optimizer = "Adam", criterion = "BCEWithLogitsLoss", seed = 1234, batch_size = 64)

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000
Unique tokens in LABEL vocabulary: 2


In [6]:
trainer.train(max_epochs = 50, improving_limit = 2, dump_id = "")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-778479ef5636>", line 1, in <module>
    trainer.train(max_epochs = 50, improving_limit = 2, dump_id = "")
  File "/home/jupyter/wl_research/wl_research_challenge/src/model.py", line 598, in train
    train_loss, train_acc = self.train_step(self.dataset["train_iterator"])
  File "/home/jupyter/wl_research/wl_research_challenge/src/model.py", line 536, in train_step
    predictions = self.model(batch.text).squeeze(1)
  File "/opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py", line 532, in __call__
    result = self.forward(*input, **kwargs)
  File "/home/jupyter/wl_research/wl_research_challenge/src/model.py", line 253, in forward
    _, hidden = self.rnn(embedded)
  File "/opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py", line 532, in __ca

KeyboardInterrupt: 

In [91]:
trainer.test()

Test Loss: 0.692 | Test Acc: 52.21%


In [ ]:
predict = trainer.get_predict_sentiment() 

In [ ]:
print(predict(sentence = "What can you do with the fire ?"))